## Feed-forward Neural Networks
In this note book we develop a system to recognize handwritten digit MNIST using feed-forward nets (FFN) a.k.a fully-connected-nets. The architecture of FFN with one hidden layer is illustrated below

<img src="./assets/ffn_1h.png" width="60%">

where in mathematics term we define
* $W_{ih}\in\mathbb{R}^{784\times H}$ is the weights from input-to-hidden
* $b_{h}\in\mathbb{R}^{H}$ is the biases for hidden layer
* $W_{ho}\in\mathbb{R}^{H\times 10}$ is the weights from hidden to output
* $b_{o}\in\mathbb{R}^{10}$ is the biases for output layer
* $a_h(x)$ is activation at hidden layer
* given an input $x$ (represented in row-vector form), the hidden layer $h$ and output layer $o$ are computed by following equation
    $$\left\{\begin{split} 
    h &= a_h\left(x\times W_{ih} + b_h\right)\\
    o &= \mathrm{softmax}\left(h\times W_{ho} + b_o\right)
    \end{split}
    \right.$$
   where 
   $$
   \mathrm{softmax}\left(v_1,\ldots,v_n\right) = \left(\frac{e^{v_1}}{\sum_{i=1}^n e^{v_i}},\ldots,\frac{e^{v_n}}{\sum_{i=1}^n e^{v_i}}\right)
   $$
* given the label $y$, the cross-entropy loss is defined as
$$
-\log(o_{y})
$$

The activation function can be one of the following forms
* $a_h(x) = \frac{1}{1+e^{-x}}$  is called sigmoid activation
* $a_h(x) = \tanh(x)$  is called tanh activation
* $a_h(x) = \max(x,0)$ is call ReLU activation

## Implement FFN